# 18.S190/6.S090 Problem Set 4

Due Friday 3/7 at **11:59pm**; 20% penalty if it is turned in within 24 hours, and after that late psets will not be accepted.   Submit in PDF format: a decent-quality scan/image of any handwritten solutions (e.g. get a scanner app on your phone or use a tablet), combined with a PDF printout of your Jupyter notebook showing your code and (clearly labeled) results.

## Problem 1

To do (Steven):

implement h-adaptive 1d quadrature with fixed-order Clenshaw–Curtis rule (from pset 3), and compare to p-adaptive.

Test out the example function from [this post](https://discourse.julialang.org/t/insanely-higher-order-derivatives/104455/17?u=stevengj), which illustrates that the adaptivity beats exponential convergence for a function with a sharp peak.

Maybe try out a couple of other functions: a smooth highly oscillatory function (where p-adaptive should be better), and a function with an endpoint singularity.

## Problem 2

To do (Qiqi):

Monte-Carlo integration for the volume of a hypersphere in d dimensions (for some set of d's).   Show $1/\sqrt{N}$ convergence.

## Problem 3

To do: something with Richardson extrapolation?  FNC book has some problems involving extrapolation methods, I think?

## Problem 4:

To do (Steven): Newton methods, from [problem 3 of this pset](https://github.com/mitmath/18335/blob/spring21/psets/pset1.pdf)